In [1]:
%pip install neattext

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,GlobalMaxPooling1D,Input
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [3]:
data=pd.read_csv('Depression_Text.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Depression_Text.csv'

In [8]:
data['label'].value_counts()

label
0.0    6259
1.0    1227
Name: count, dtype: int64

In [9]:
data['label'].value_counts().index.values

array([0., 1.])

In [10]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [11]:
train_data['label'].value_counts().index.values

array([0., 1.])

In [12]:
labels = train_data['label'].value_counts()
plx.bar(x=labels.index.values, y=labels.values, color=[0,1])

In [13]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [ ]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

In [ ]:
# Check the data type of the 'text' column in the 'train_data' and 'test_data' DataFrames.
# If it is not a string, convert it to a string.
train_data['text'] = train_data['text'].astype(str)
test_data['text'] = test_data['text'].astype(str)

# Clean the text in the 'train_data' and 'test_data' DataFrames.
cleaned_train_text, train_text_length = clean_text(train_data)

In [ ]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

In [21]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [ ]:
cleaned_train_text

In [22]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=50)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=50)

In [ ]:
train_text_pad

In [24]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['label'])
test_output=lbl_target.transform(test_data['label'])

In [ ]:
!pip install pickle-mixin

In [26]:
!wget https://nlp.stanford.edu/data/glove.840B.300d.zip

--2024-04-06 11:11:17--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2024-04-06 11:11:17--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  5.14MB/s    in 6m 53s  

2024-04-06 11:18:11 (5.02 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [27]:
import zipfile

# Specify the path to the zip file
zip_file_path = "/content/glove.840B.300d.zip"

# Specify the directory to extract the files to
target_directory = "/content"

# Open the zip file
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    # Extract all the files in the zip file to the target directory
    zip_ref.extractall(target_directory)

In [ ]:
import numpy as np

def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                vector = np.array(values[1:], dtype='float32')
                embeddings[word] = vector
            except ValueError:
                print(f"Issue parsing line: {line}")
    return embeddings

file_path = '/content/glove.840B.300d.txt'
glove_embedding = load_glove_embeddings(file_path)


In [29]:
v=len(tokenizer.word_index)

embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.074482  ,  0.58293003, -0.78233999, ..., -0.24984001,
        -0.096953  ,  0.66692001],
       [ 0.065244  ,  0.14688   , -0.30096999, ..., -0.24393   ,
         0.35209   ,  0.24132   ],
       ...,
       [-0.10076   , -0.18752   , -0.053066  , ...,  0.015689  ,
         0.27893999,  0.50827998],
       [ 0.79620999, -0.18398   , -0.052325  , ...,  0.80081999,
        -0.82199001, -0.40882   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [30]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

In [31]:
model=Sequential()
model.add(Input(shape=(40,)))
model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
model.add(LSTM(20,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.SGD(0.1,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=256,callbacks=[early_stop,reducelr])

In [34]:
print(train_text_pad.shape)
print(test_text_pad.shape)

(5991, 50)
(1498, 50)


In [35]:
model = Sequential()
model.add(Input(shape=(50,)))
model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
model.add(LSTM(20,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.SGD(0.1,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [37]:
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=256,callbacks=[early_stop,reducelr])

Epoch 1/20
24/24 [==============================] - 16s 122ms/step - loss: 0.5329 - accuracy: 0.8241 - val_loss: 0.4599 - val_accuracy: 0.8385 - lr: 0.1000
Epoch 2/20
24/24 [==============================] - 2s 80ms/step - loss: 0.4522 - accuracy: 0.8351 - val_loss: 0.4437 - val_accuracy: 0.8385 - lr: 0.1000
Epoch 3/20
24/24 [==============================] - 2s 85ms/step - loss: 0.4407 - accuracy: 0.8351 - val_loss: 0.4327 - val_accuracy: 0.8385 - lr: 0.1000
Epoch 4/20
24/24 [==============================] - 2s 84ms/step - loss: 0.4295 - accuracy: 0.8351 - val_loss: 0.4190 - val_accuracy: 0.8385 - lr: 0.1000
Epoch 5/20
24/24 [==============================] - 3s 143ms/step - loss: 0.4128 - accuracy: 0.8351 - val_loss: 0.3992 - val_accuracy: 0.8385 - lr: 0.1000
Epoch 6/20
24/24 [==============================] - 4s 162ms/step - loss: 0.3901 - accuracy: 0.8351 - val_loss: 0.3733 - val_accuracy: 0.8385 - lr: 0.1000
Epoch 7/20
24/24 [==============================] - 4s 160ms/step - loss

In [ ]:
print('TESTING DATA CLASSIFICATION REPORT \n \n')
print(classification_report(test_output,model.predict_classes(test_text_pad),
                            target_names=lbl_target.inverse_transform([0,1])))

print('TRAINING DATA CLASSIFICATION REPORT \n \n')
print(classification_report(train_output,model.predict_classes(train_text_pad),
                           target_names=lbl_target.inverse_transform([0,1])))

In [39]:
# Print the shapes of the train and test text padding
print(train_text_pad.shape)
print(test_text_pad.shape)

# Define the model
model = Sequential()
model.add(Input(shape=(50,)))
model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
model.add(LSTM(20,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.SGD(0.1,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

# Print the model summary
model.summary()

# Fit the model
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=256,callbacks=[early_stop,reducelr])

# Print the testing data classification report
print('TESTING DATA CLASSIFICATION REPORT \n \n')
print(classification_report(test_output, np.argmax(model.predict(test_text_pad), axis=1)))

(5991, 50)
(1498, 50)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 300)           3099900   
                                                                 
 lstm_2 (LSTM)               (None, 50, 20)            25680     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 20)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 256)               5376      
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 3131213 (11.94 MB)
Trainable params: 31313 (122.32 KB)
Non-trainable params: 3099900

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [40]:
# Change this line:
predicted_classes = (model.predict(test_text_pad))

# To this:
predicted_classes = np.argmax(model.predict(test_text_pad), axis=1)

# And change this line:
predicted_classes = (model.predict(train_text_pad))

# To this:
predicted_classes = np.argmax(model.predict(train_text_pad), axis=1)

188/188 [==============================] - 5s 25ms/step


In [41]:
twt = ['let Gather For Party']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)

prediction = model.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("depression")
else:
    print("not")

1/1 [==============================] - 0s 97ms/step
0.01106835
not


In [42]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
token_form = pickle.load(open('tokenizer.pkl', 'rb'))

In [ ]:
from keras.models import load_model

In [ ]:
model_form = load_model("model.h5")

In [ ]:
twt = ['Through these past years thoughts of suicide, fear, anxiety I’m so close to my limit']
twt = token_form.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)


prediction = model_form.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("depression")
elif (prediction == 1):
    print("not")

1/1 [==============================] - 1s 1s/step
0.9474559
depression


In [ ]:

!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2ehP6P3OPSRKJuXWe5On98jOvDs_2eVucQSJzFYm7AjRbL3GQ

In [ ]:
!ngrok -h


In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return "<h1>Heo, wld!</h1>"

if __name__ == "__main__":
    app.run()
